In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import math

from glob import glob
from PIL import Image
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from time import sleep

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

## Make Label

In [4]:
df = pd.read_csv('/opt/ml/code/data/Final.csv')
df.head()

,id,path,age,gender,stem,img_path,gender_issue,mask_issue
0,000001,000001_female_Asian_45,45,female,mask1,/opt/ml/input/data/train/images/000001_female_...,False,False
1,000001,000001_female_Asian_45,45,female,mask2,/opt/ml/input/data/train/images/000001_female_...,False,False
2,000001,000001_female_Asian_45,45,female,mask4,/opt/ml/input/data/train/images/000001_female_...,False,False
3,000001,000001_female_Asian_45,45,female,mask3,/opt/ml/input/data/train/images/000001_female_...,False,False
4,000001,000001_female_Asian_45,45,female,incorrect_mask,/opt/ml/input/data/train/images/000001_female_...,False,False


In [5]:
df['label'] = ''
for i in range(len(df)) :
    label = 0
    # 나이를 기준으로 점수
    if df['age'][i] < 30 :
        label = 0
    elif 30 <= df['age'][i] < 60 :
        label = 1
    elif 60 <= df['age'][i] :
        label = 2
    
    # 여자일 경우 남자의 label +3
    if df['gender'][i] == 'female' :
        label += 3
    
    # mask상태가 Not wear일 경우 + 12, Incorrect일 경우 + 6
    if df['stem'][i] == 'normal' :
        label += 12
    elif df['stem'][i] == 'incorrect_mask' :
        label += 6
    
    df['label'][i] = label

In [6]:
df.sample(10)

,id,path,age,gender,stem,img_path,gender_issue,mask_issue,label
2471,000799,000799_female_Asian_51,51,female,mask1,/opt/ml/input/data/train/images/000799_female_...,False,False,4
1289,000566,000566_male_Asian_58,58,male,mask2,/opt/ml/input/data/train/images/000566_male_As...,False,False,1
1907,000684,000684_female_Asian_56,56,female,mask3,/opt/ml/input/data/train/images/000684_female_...,False,False,4
9165,003328,003328_female_Asian_19,19,female,mask4,/opt/ml/input/data/train/images/003328_female_...,False,False,3
851,000304,000304_female_Asian_51,51,female,incorrect_mask,/opt/ml/input/data/train/images/000304_female_...,False,False,10
7379,001922,001922_male_Asian_42,42,male,mask2,/opt/ml/input/data/train/images/001922_male_As...,False,False,1
15839,006065,006065_female_Asian_19,19,female,normal,/opt/ml/input/data/train/images/006065_female_...,False,False,15
14446,005221,005221_male_Asian_21,21,male,mask5,/opt/ml/input/data/train/images/005221_male_As...,False,False,0
11187,003760,003760_female_Asian_52,52,female,mask2,/opt/ml/input/data/train/images/003760_female_...,False,False,4
1792,000661,000661_male_Asian_55,55,male,mask1,/opt/ml/input/data/train/images/000661_male_As...,False,False,1


## Dataset Load

In [7]:
class MaskDataset(Dataset) :
    def __init__(self, path_list, label_list, transform, train = True) :
        self.train = train
        if self.train :
            self.X = []
            self.y = []
            for path, label in zip(path_list, label_list) :
                image = Image.open(path)
                self.X.append(image)
                self.y.append(label)
        else :
            self.X = []
            for path in path_list :
                image = Image.open(path)
                self.X.append(image)
        
        self.transform = transform
        self._repr_indent = 4
        if self.train :
            self.classes = list(set(self.y))
    
    def __len__(self) :
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx) :
        X = self.X[idx]
        X = self.transform(X)
        if not self.train :
            y = None
        else :
            y = self.y[idx]
        return torch.tensor(X, dtype = torch.float), torch.tensor(y, dtype = torch.long)
    
    def __repr__(self) :
        head = "(PyTorch Practice) My Custom Dataset : MASK"
        num_data = self._repr_indent*" " + "Number of datapoints : {}".format(self.__len__())
        
        if self.train :
            num_classes = self._repr_indent*" " + "Number of classes {}".format(len(self.classes))
        else :
            num_classes = self._repr_indent*" " + "Number of classes None"
            
        return '\n'.join([head, num_data, num_classes])

In [8]:
data_path = df['img_path']
target = list(df['label'])

### ResNet 학습

In [12]:
# Resnet transfer learniong
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters() : # frozon
    param.requires_grad = False
    
MASK_CLASS_NUM = 18
model.fc = torch.nn.Linear(in_features = 512, out_features = 128, bias = True)
model.fc.weight.requires_grad = True
torch.nn.init.xavier_uniform_(model.fc.weight)
stdv = 1.0 / np.sqrt(model.fc.in_features)
model.fc.bias.data.uniform_(-stdv,stdv)

model.relu1 = torch.nn.ReLU(inplace = True)
model.drop1 = torch.nn.Dropout(p = 0.2)
model.fc2 = torch.nn.Linear(128, 32, bias = True)
model.relu2 = torch.nn.ReLU(inplace = True)
model.drop2 = torch.nn.Dropout(p = 0.2)
model.fc3 = torch.nn.Linear(32, 18, bias = True)

In [13]:
model.conv1.weight.requires_grad = True
model.bn1.weight.requires_grad = True
#model.relu.weight.requires_grad = True
#model.maxpool.weight.requires_grad = True
model.layer1[0].conv1.weight.requires_grad = True
model.layer1[0].bn1.weight.requires_grad = True
#model.layer1[0].relu.weight.requires_grad = True
model.layer1[0].conv2.weight.requires_grad = True
model.layer1[0].bn2.weight.requires_grad = True
model.layer1[1].conv1.weight.requires_grad = True
model.layer1[1].bn1.weight.requires_grad = True
#model.layer1[1].relu.weight.requires_grad = True
model.layer1[1].conv2.weight.requires_grad = True
model.layer1[1].bn2.weight.requires_grad = True
model.layer2[0].conv1.weight.requires_grad = True
model.layer2[0].bn1.weight.requires_grad = True
#model.layer1[0].relu.weight.requires_grad = True
model.layer2[0].conv2.weight.requires_grad = True
model.layer2[0].bn2.weight.requires_grad = True
model.layer2[1].conv1.weight.requires_grad = True
model.layer2[1].bn1.weight.requires_grad = True
#model.layer1[1].relu.weight.requires_grad = True
model.layer2[1].conv2.weight.requires_grad = True
model.layer2[1].bn2.weight.requires_grad = True

In [ ]:
stf = StratifiedKFold(n_splits = 4, shuffle = True, random_state =42)
foldperf = {}
for fold, (train_idx, valid_idx) in enumerate(stf.split(data_path, target)) :
    
    print('Fold {}'.format(fold + 1))
    target_array = np.array(target)
    
    dataset_train_Mask = MaskDataset(path_list = data_path[train_idx],
                                     label_list = target_array[train_idx],
                                     transform = transforms.Compose([
                                         Resize((512, 384), Image.BILINEAR),
                                         ToTensor(),
                                         Normalize(mean = (0.5,0.5,0.5), std = (0.2, 0.2, 0.2)),
                                      ]),
                                      train = True,
                                      )
    dataset_valid_Mask = MaskDataset(path_list = data_path[valid_idx],
                                     label_list = target_array[valid_idx],
                                     transform = transforms.Compose([
                                          Resize((512, 384), Image.BILINEAR),
                                          ToTensor(),
                                          Normalize(mean = (0.5,0.5,0.5), std = (0.2, 0.2, 0.2)),
                                      ]),
                                      train = True,
                                      )
    BATCH_SIZE = 256
    mask_train_dataloader = torch.utils.data.DataLoader(dataset_train_Mask,
                                                        batch_size = BATCH_SIZE,
                                                        shuffle = True)
    mask_valid_dataloader = torch.utils.data.DataLoader(dataset_valid_Mask,
                                                        batch_size = BATCH_SIZE,
                                                        shuffle = True)
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'{device} is using !')
    sleep(1)
    
    model = model.to(device)

    LEARNING_RATE = 0.0001
    NUM_EPOCH = 100

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

    dataloaders = {
        'train' : mask_train_dataloader,
        'test' : mask_valid_dataloader
    }
    
    history = {'train_loss' : [], 'test_loss' : [],
               'train_acc' : [], 'test_acc' : [],
               'train_f1' : [], 'test_f1' : []}
    
    n_epochs_stop = 3
    epochs_no_improve = 0
    early_stop = False
    min_val_loss = np.Inf
    
    best_test_accuracy = 0
    best_test_loss = 9999.
    for epoch in range(20) :
        for phase in ['train', 'test'] :
            running_loss = 0.
            running_acc = 0.
            running_f1 = 0.
            n_iter = 0
            
            if phase == 'train' :
                model.train()
            elif phase == 'test' :
                model.eval()

            for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])) :
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train') :
                    logits = model(images)
                    _, preds = torch.max(logits, 1)
                    loss = loss_fn(logits, labels)

                    if phase == 'train' :
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * images.size(0)
                running_acc += torch.sum(preds == labels.data)
                running_f1 += f1_score(preds.cpu().numpy(), labels.cpu().numpy(), average = 'macro')
                n_iter += 1

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_acc / len(dataloaders[phase].dataset)
            epoch_f1 = running_f1 / n_iter
            #epoch_f1_check = running_f1 / len(dataloaders[phase].dataset)
            #print('n_iter는 ', n_iter, ' dataset는 :', len(dataloaders[phase].dataset))
            #print('n_iter :', epoch_f1, ' dataset :', epoch_f1_check)
            
            if phase == 'test' :
                if epoch_loss < min_val_loss :
                    epochs_no_improve = 0
                    min_val_loss = epoch_loss
                else :
                    epochs_no_improve += 1
                
                if epochs_no_improve == n_epochs_stop :
                    print('Early Stopping!')
                    early_stop = True
                    break
            
            if phase == 'train' :
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc)
                history['train_f1'].append(epoch_f1)
            elif phase == 'test' :
                history['test_loss'].append(epoch_loss)
                history['test_acc'].append(epoch_acc)
                history['test_f1'].append(epoch_f1)

            print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.4f}, 평균 Accuracy : {epoch_acc:.4f}, 평균 F1 Score : {epoch_f1: .4f}")
            #if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
            #    best_test_accuracy = epoch_acc
            #if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
            #    best_test_loss = epoch_loss
    
        if early_stop :
            print(f'fold{fold+1} Stopped')
            break
    foldperf['fold{}'.format(fold+1)] = history 
    
    #print("학습 종료!")
    #print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

Fold 1
cuda:0 is using !


  0%|          | 0/19 [00:00<?, ?it/s]

현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.8723, 평균 Accuracy : 0.7752, 평균 F1 Score :  0.4864


  0%|          | 0/56 [00:00<?, ?it/s]

현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.8107, 평균 Accuracy : 0.7964, 평균 F1 Score :  0.5254


...

  0%|          | 0/56 [00:00<?, ?it/s]

현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.0039, 평균 Accuracy : 1.0000, 평균 F1 Score :  1.0000


  0%|          | 0/19 [00:00<?, ?it/s]

현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.0041, 평균 Accuracy : 1.0000, 평균 F1 Score :  1.0000


 79%|███████▉  | 15/19 [00:15<00:04,  1.04s/it]

In [96]:
torch.save(model, '/opt/ml/code/model/data-modify-resnet18-freezen-more-two-layer-linear-SKFold4-earlystopping.pt')

In [97]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [98]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

In [99]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [100]:
# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

In [101]:
device = torch.device('cuda')
model = torch.load('/opt/ml/code/model/data-modify-resnet18-freezen-more-two-layer-linear-SKFold4-earlystopping.pt').to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [102]:
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

100%|██████████| 12600/12600 [02:06<00:00, 99.62it/s] 


In [103]:
# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_fifth.csv'), index=False)
print('test inference is done!')

test inference is done!
